In [40]:
import requests as reqs, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

def print_summary_report(consultant_name = 'Ryan', fromToList =[]):
    '''
    Prints a CLI summary report for each consultant (last 30 days)
    '''

    def consultant_dataframe():
        '''
        Setting up pandas DF to store consultant identifiers
        '''
        consultant_df = pd.DataFrame( # creating dataframe to hold employee info
            data= {
                'API Key': [
                    'NjgzYjZmODUtZmQwOC00OWNiLWE2N2QtYTU3OTNlYjhiZjA0',
                    'MTllMGE3ZGUtNzdlNS00OTJmLWEwNGQtOGE3N2NiOWFlMmZm',
                    'MGE1YmU3NzgtNGEyMi00MDdkLTgwMmMtMGQ5MWY2OWY3Mjhj',
                    'ZmQ4M2UwNTgtNzVkZC00NTgwLTgyZjktZjc3ZmYyN2U3MDA0',
                    'NDQ3NTUwY2YtMjNkZi00YjQ2LThkNTMtZDhlY2Y0NzYyMTQ4',
                    'YWY1MTMxYWItZGVjNC00YWJhLWE2MjgtMmQxZDM1ZThiYmYw'
                    ],
                'ID': [
                    '617ff508d96e4e121a93f17a',
                    '60aeec0f215fef1f3a4e0f58',
                    '61d5ddc696aafe5141d19e2f',
                    '6238b01a3e89a17fb517d7dd',
                    '62558a0a5daa063d7fc96423',
                    '62263af2473a946e318e9c2a'
                    ]
                },
            index= [
                'Chris Lowden',
                'Ryan Moore',
                'Gerry Mattei',
                'Innes Tsui',
                'Scott Kolmar',
                'Brennan Coles'
            ]
        )
        return consultant_df

    consultant_df = consultant_dataframe() # Initialize the consultant DF
    #date_range = handle_dates()
        
    API, ID  = consultant_df.loc[consultant_name,'API Key'] , consultant_df.loc[consultant_name,'ID']
    
    
    r = reqs.post( # creating the request object
        url= f'https://reports.api.clockify.me/v1/workspaces/{ID}/reports/summary', #static URL for the summary endpoint
        headers = { # headers for summary request
            "content-type": "application/json", 
            "X-Api-Key":API
        }, 
        json = { # JSON payload to specify additional parameters
            "dateRangeStart": str(fromToList[0]),
            "dateRangeEnd": str(fromToList[1]),
            "summaryFilter": {"groups": ["PROJECT"]},
            "sortOrder": "DESCENDING"
        }
    )
    
    json2dict = dict(r.json())
    df2 = pd.DataFrame(json2dict['groupOne'])
    df2['duration'] /=3600
    df2['Consultant_Name'] = consultant_name
    df2.drop(['amounts','amount','name','nameLowerCase','color'], axis=1, inplace=True)
    
    
    rates_df = pd.read_excel(r'C:\Users\mryan\Desktop\Rates.xlsx')
    #rates_df.dropna(inplace=True)
    rates_df.set_index('User', inplace=True)
    
    #df2.join(rates_df, on='Consultant_Name', how='left')
    df3 = df2.merge(rates_df, how = 'left', left_on=['Consultant_Name', 'clientName'], right_on = ['User','Client'])
    #df3.dropna(inplace=True)
    df3.drop(['_id','clientName','Pay_Rate','To','cc'], axis=1, inplace=True)
    df3.dropna(inplace=True)
    for i in df3:
        df3['Bill_Total'] = df3['duration'] * df3['Bill_Rate']
    
    #print('\n' + consultant_name)
    #display(df3)
    
    return df3
    
    
    #print('\n' + consultant_name)
    #print(df2.loc[0:,['clientName', 'duration']].groupby('clientName').sum())
    #print('--------------------------------------------------------------------------------')



In [43]:
def handle_dates(fromYear= 2022, fromMonth= 1, fromDay= 1, months2Add = 1):
    '''
    Expanding the date selection capabilities for clockify POST req.
    '''
    fromdate = datetime(fromYear,fromMonth,fromDay)
    rd = relativedelta(months=months2Add)
    
    return [fromdate.strftime("%Y-%m-%dT%H:%M:%S.%f"), (fromdate+rd).strftime("%Y-%m-%dT%H:%M:%S.%f")]

def report_printer(
        fYear= 2022,
        fMonth= 1,
        fDay= 1, 
        addMonths = 1,
        names = [
            'Chris Lowden',
            'Ryan Moore',
            'Gerry Mattei',
            'Innes Tsui',
            'Scott Kolmar',
            'Brennan Coles'
            ] ):
    
    fromdate = datetime(fYear,fMonth,fDay)
    
    rd = relativedelta(months=addMonths)
    
    
    #print('--------------------------------------------------------------------------------')
    #print('WFI Consulting Hours Summary' +f"\n From: {str(fromdate)}" + f"\n To: {str(fromdate+rd)}")
    #print('--------------------------------------------------------------------------------')
    
    dfList = []
    for i in names:
        report_df_temp = print_summary_report(i, handle_dates(fYear,fMonth,fDay, addMonths))
        dfList.append(report_df_temp)
    
    # dfList = [print_summary_report(i, handle_dates(fYear,fMonth,fDay, addMonths)) for i in names]
    # return dfList

    return dfList
    
totals_df = pd.concat(report_printer(2022,7,1,1))
totals_df.groupby(['Client']).sum()

,duration,Bill_Rate,Bill_Total
Client,,,
DG Medicines,6.00,200.0,1200.00
Eikonizo,19.25,470.0,1766.25
Elanco,6.00,200.0,1200.00
Engine,18.50,350.0,3712.50
Gilgamesh,20.25,425.0,3768.75
Halda,49.50,550.0,6750.00
Ideaya,29.75,425.0,5043.75
Kymera,2.00,200.0,400.00
LifeMine,66.25,250.0,6643.75


In [33]:
rates_df = pd.read_excel(r'C:\Users\mryan\Desktop\Rates.xlsx')
rates_df.dropna(inplace=True)

rates_df.loc(['User'][0:] == 'Scott Kolmar')
#rates_df['User'][1]

In [11]:
for i in 

Signature:
rates_df.dropna(
    axis: 'Axis' = 0,
    how: 'str' = 'any',
    thresh=None,
    subset: 'IndexLabel' = None,
    inplace: 'bool' = False,
)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. versionchanged:: 1.0.0

       Pass tuple or list to drop on multiple axes.
       Only a single axis is allowed.

how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.

thresh : int